<a href="https://colab.research.google.com/github/rcleveng/notebooks/blob/main/Complete_(No_DB)_Manual_RAG_with_Google_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple RAG langchain example using Google's Generate AI API.
This example let's you ask questions against a book.

You'll need an API key from Generative AI Studio.


In [ ]:
!pip install langchain google.generativeai langchain-google-genai ratelimit more_itertools


# Restart the kernel once dependencies are updated

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [3]:
# @title Set the book URL to use for Q&A

# Change this to use a different book.
BOOK_URL="https://www.gutenberg.org/cache/epub/3300/pg3300.txt" #@param {type:"string"}

In [4]:
# @title Fetch and split URL into chunks

import requests
from google.colab import userdata

# Langchain does a reasonable job splitting easily, so we'll use that

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document


# Get text from URL
def get_data(url):
 r = requests.get(url)
 return r.text

BOOK_TEXT = get_data(BOOK_URL)
doc =  Document(page_content=BOOK_TEXT, metadata={"source": "local"})
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents([doc])

# Count the number of chunks
print ('Finished splitting document into ', len(texts), 'chunks.')


Finished splitting document into  3102 chunks.


In [5]:
import pandas as pd
import google.generativeai as genai
from ratelimit import limits, sleep_and_retry
from google.colab import userdata
from more_itertools import batched
import requests

clean_texts = list(map(lambda x: x.page_content, texts))
google_api_key = userdata.get('PALM_KEY')
genai.configure(api_key=google_api_key)

Count = 1
@sleep_and_retry
@limits(calls=1500, period=60)
def batch_embed_fn(text):
  global Count
  Count += 1
  if Count % 10 == 0:
    print("Batch", Count)
  return genai.embed_content(model="models/embedding-001",
                             content=text,
                             task_type="retrieval_document")["embedding"]


embeddings = []
for batch in batched(clean_texts, 50):
  emb_batch = batch_embed_fn(batch)
  embeddings.extend(emb_batch)


d = { 'id' : range(0, len(clean_texts)),
      'text' : clean_texts,
      'embeddings' : embeddings }
df = pd.DataFrame(d)

print('Dataframe created for ', len(clean_texts), ' items.')


Batch 10
Batch 20
Batch 30
Batch 40
Batch 50
Batch 60
Dataframe created for  3102  items.


In [6]:
df


,id,text,embeddings
0,0,﻿The Project Gutenberg eBook of An Inquiry int...,"[0.04775882, -0.010786952, -0.05073722, -0.034..."
1,1,An Inquiry into the Nature and Causes of the W...,"[0.04635505, -0.02614058, -0.029601555, 0.0144..."
2,2,CHAPTER VII. OF THE NATURAL AND MARKET PRICE O...,"[0.048168775, -0.00994209, -0.0056164246, 0.02..."
3,3,DIFFERENT NATIONS\r\n CHAPTER I. OF THE NATURA...,"[0.058188085, -0.022204645, -0.034955118, 0.04..."
4,4,CHAPTER IV. OF DRAWBACKS.\r\n CHAPTER V. OF BO...,"[0.058553644, -0.013853595, -0.03506425, -0.01..."
...,...,...,...
3097,3097,Sections 3 and 4 and the Foundation informatio...,"[0.026205279, -0.009810904, -0.04124547, -0.05..."
3098,3098,and official page at www.gutenberg.org/contact...,"[0.03730992, -0.002867783, -0.055633023, -0.05..."
3099,3099,with these requirements. We do not solicit don...,"[0.027048666, -0.006232095, -0.06311451, -0.02..."
3100,3100,methods and addresses. Donations are accepted ...,"[0.030136747, -0.01517519, -0.060330722, -0.05..."


In [10]:
# @title Ask Questions on The Wealth of Nations
import numpy as np
from numpy.linalg import norm


# Ask question
question = "what is capital?" #@param {type:"string"}

# emedded query
emb_question = genai.embed_content(model="models/embedding-001",
                                   content=question,
                                   task_type="retrieval_query")["embedding"]

print("emb_question: ", type(emb_question), emb_question)


# Compute cosine similarity
cosine_similarities = np.dot(embeddings, emb_question) / (norm(embeddings) * norm(emb_question))

# Find the index of the most similar sentence
most_similar_index = np.argmax(cosine_similarities)
print(f"\n\n***********************\nMost similar sentence: {clean_texts[most_similar_index]}")

topk = np.argsort(cosine_similarities)[-5:]
for k in topk:
  print('\n\n***********************\nArticle id: ', k, ': ', clean_texts[k])




context = "\n".join(map(lambda k: clean_texts[k], topk))

prompt = f"""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try
to make up an answer.

{context}

Question: {question}
Helpful Answer:

"""

print("\n\n***********************\nUsing full prompt: ", prompt)
answer = ""

model_name = 'gemini-pro' # @param ["gemini-pro", "models/text-bison-001"] {allow-input: false}

# emedded query
if model_name == "gemini-pro":
  model = genai.GenerativeModel('gemini-pro')
  answer = model.generate_content(question).text
else:
  c = genai.generate_text(model=model_name, prompt=question, max_output_tokens=2000)
  answer = c.result

print("answer: ", answer)


emb_question:  <class 'list'> [0.023361903, -0.061758768, -0.035787556, -0.015796369, 0.048259538, 0.024039365, -0.0120574655, -0.02556999, 0.02532436, 0.00065946154, 0.017345952, 0.009818537, -0.051182028, -0.012903035, 0.027620565, -0.027579844, 0.014208836, 0.057374287, 0.056737628, -0.033245727, -0.010844937, 0.01336066, -0.02102469, 0.02137933, -0.026187263, -0.017910024, 0.029666174, -0.08724266, -0.00700557, 0.06243666, -0.075376764, -0.011758794, -0.028740099, 0.023746116, 0.03211378, -0.0136165945, 0.03879212, 0.028489407, 0.048188444, 0.025664853, -0.023164926, 0.0043488583, -0.07904044, 0.012449268, -0.00040266232, -0.016651802, 0.036149096, -0.031368263, 0.0077752406, -0.050365347, 0.039318286, 0.0019275616, 0.036187425, 0.0042401133, -0.009630864, -0.03121317, 0.0492909, -0.005665149, -0.06655225, -0.011374893, 0.009631971, 0.027822645, -0.021413673, 0.06529165, 0.046849713, 0.0036123372, -0.024301812, 0.05115978, 0.06576924, -0.01680737, -0.030509325, -0.028676502, 0.0969